## Loading data in the database
---
### We will perform the following steps in order to load the data into mongoDB

1. read the cleaned csv data i.e `matches.csv`
2. iterate through the data and make http request to the backend API

### Data structure in the backend
**In the backend we have 2 tables**
1. teams
2. matches

`matches` table is has the same structure as the `matches.csv`, but for `teams` table we have to perform some transformation on the data we have

Cool! Let's get started with matches table

In [1]:
import pandas as pd
import requests
import typing
import json
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_matches = pd.read_csv("matches.csv")

In [3]:
df_matches.head(2)

,id,city,date,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,umpire1,umpire2
0,335982,Bengaluru,2008-04-18,BB McCullum,M.Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,MR Benson,SL Shastri


In [4]:
_API_URL = "http://localhost:4000"

headers = {
    "Content-Type": "application/json"
}

In [5]:
resp = requests.get(_API_URL)
print(resp.json(),resp.status_code)

{'info': 'IPL Dashboard backend API', 'status': 'active'} 200


In [6]:
def create_match(match:typing.Dict) -> typing.Union[typing.Dict,None]:
    """
    Calls the backend API to create new match
    """
    try:
        resp = requests.post(_API_URL+"/matches",headers=headers,data = json.dumps(match))
        if resp.status_code == 201:
            return resp.json()
        else:
            print(resp.content)
    except Exception as e:
        print(e)
        return None

In [7]:
failed_for = []

for index,match in df_matches.iterrows():
    new_match = create_match(match.to_dict())
    if new_match:
        pass
    else:
        failed_for.append(match.to_dict())

print(failed_for)    

[]


### teams table

**Teams table has the following structure**
```
{
    name: {
      type: String,
      required: true,
      trim: true,
      unique: true,
    },
    total_matches: {
      type: Number,
      required: true,
    },
    total_wins: {
      type: Number,
      required: true,
    },
  }
```

As you can see we have to perform some data prepration before calling the backend API

In [8]:
def get_info(team_name:str,df:pd.DataFrame)-> typing.Union[typing.Dict,None]:
    """
    calculates total number of matches and total number of wins for a team.
    """
    try:
        return {
            "name":team_name,
        }
    except Exception as e:
        print(e)
        return None

In [9]:
# test

get_info("Royal Challengers Bangalore",df_matches)

{'name': 'Royal Challengers Bangalore'}

In [10]:
def create_team(team:typing.Dict) -> typing.Union[typing.Dict,None]:
    """
    Calls the backend API to create new team
    """
    try:
        resp = requests.post(_API_URL+"/teams",headers=headers,data = json.dumps(team))
        if resp.status_code == 201:
            return resp.json()
        else:
            print(resp.content)
    except Exception as e:
        print(e)
        return None

In [11]:
# iterate over unique team
unique_team_names = list(set(df_matches["team1"].tolist() + df_matches["team2"].tolist()))

In [12]:
unique_team_names

['Kings XI Punjab',
 'Kolkata Knight Riders',
 'Royal Challengers Bangalore',
 'Gujarat Lions',
 'Mumbai Indians',
 'Rajasthan Royals',
 'Sunrisers Hyderabad',
 'Chennai Super Kings',
 'Rising Pune Supergiant',
 'Kochi Tuskers Kerala',
 'Delhi Capitals']

In [13]:
for team in unique_team_names:
    team_payload = get_info(team,df_matches)
    new_team = create_team(team_payload)
    if new_team:
        print("created!!")
    else:
        print("failed")

created!!
created!!
created!!
created!!
created!!
created!!
created!!
created!!
created!!
created!!
created!!


## Cool! we have uploaded all the data

In [15]:
df_matches[df_matches["winner"] == "Kolkata Knight Riders"].shape

(97, 14)